In [8]:
!pip install neo4j


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
from neo4j import GraphDatabase

URI = 'bolt://localhost:7687'
AUTH = (NEO4J_USERNAME, 'promact0')

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

In [ ]:
import Node
Nodes=[
    Node(id='deceased', type='Person'),
    Node(id='Ram Kishore Chaudhary', type='Person'),
    Node(id='Title Appeal No. 19 of 2006', type='Case'), 
    Node(id='death inquest report', type='Document'), 
    Node(id='statement of Beena Devi', type='Document'), 
    Node(id='brother of Ranjeet Choudhary', type='Person'), 
    Node(id='blood mark', type='Mark'), 
    Node(id='country made rifle', type='Weapon'), 
    Node(id='Magistrate', type='Person'), 
    Node(id='R.I. for 03 years', type='Sentence'), 
    Node(id='inquest report', type='Document'), 
    Node(id='Falka Pothiya P.S. Case No. 26 of 2008', type='Case'), 
    Node(id='Uday Kumar', type='Person'), 
    Node(id='statement of Uro Devi', type='Document'), 
    Node(id='land dispute', type='Dispute'), 
    Node(id='Suro Devi', type='Person'), 
    Node(id='Beena Devi', type='Person'), 
    Node(id='statement of Satyanarain Yadav', type='Document'), 
    Node(id='Ranjeet Choudhary', type='Person'), 
    Node(id='S.T. No. 204 of 2010', type='Case'), 
    Node(id='weapon', type='Weapon'), 
    Node(id='Satish Kumar Yadav', type='Person'), 
    Node(id='place of occurrence', type='Location'), 
    Node(id='Section 302 IPC', type='Law'), 
    Node(id='assault on his brother', type='Event'), 
    Node(id='fire arm injury', type='Injury'), 
    Node(id='palani', type='Structure'), 
    Node(id='Informant', type='Person'), 
    Node(id='Court', type='Court'), 
    Node(id='dead body of the deceased', type='Body'), 
    Node(id='Addl. District and Sessions Judge-II, Katihar', type='Court'), 
    Node(id='Anil Kumar', type='Person'), 
    Node(id='Dumar Chowk', type='Location'), 
    Node(id='Cr. Appeal (DB) No. 117 of 2013', type='Case'), 
    Node(id='wound of exit', type='Injury'), 
    Node(id='17.07.2008', type='Date'), 
    Node(id='Additional District and Sessions Judge-I, Katihar', type='Court'), 
    Node(id='Cr. Appeal (DB) No. 19 of 2013', type='Case'), 
    Node(id='fardbeyan', type='Document'), 
    Node(id='Title Suit No. 43 of 1998', type='Case'), 
    Node(id='Section 302/34 IPC', type='Law'), 
    Node(id='Mantaria Bandh', type='Location'), 
    Node(id='occurrence', type='Event'), 
    Node(id='F.I.R.', type='Document'), 
    Node(id='Section 27 of the Arms Act', type='Law'), 
    Node(id='statement of Satish Kumar Yadav', type='Document'), 
    Node(id='shell of cartridges', type='Object'), 
    Node(id='R.I. for life', type='Sentence'), 
    Node(id='Police Station', type='Location')
]

In [ ]:
import Relationship
Relationships=[
    Relationship(source=Node(id='Ranjeet Choudhary', type='Person'), target=Node(id='Section 302 IPC', type='Law'), type='CONVICTED_OF'), 
    Relationship(source=Node(id='Ranjeet Choudhary', type='Person'), target=Node(id='Section 27 of the Arms Act', type='Law'), type='CONVICTED_OF'), 
    Relationship(source=Node(id='Ram Kishore Chaudhary', type='Person'), target=Node(id='Section 302/34 IPC', type='Law'), type='CONVICTED_OF'), 
    Relationship(source=Node(id='Ranjeet Choudhary', type='Person'), target=Node(id='R.I. for life', type='Sentence'), type='SENTENCED_TO'), 
    Relationship(source=Node(id='Ram Kishore Chaudhary', type='Person'), target=Node(id='R.I. for life', type='Sentence'), type='SENTENCED_TO'), 
    Relationship(source=Node(id='Ranjeet Choudhary', type='Person'), target=Node(id='R.I. for 03 years', type='Sentence'), type='SENTENCED_TO'), 
    Relationship(source=Node(id='Ranjeet Choudhary', type='Person'), target=Node(id='Addl. District and Sessions Judge-II, Katihar', type='Court'), type='WORKS_FOR'), 
    Relationship(source=Node(id='Ram Kishore Chaudhary', type='Person'), target=Node(id='Addl. District and Sessions Judge-II, Katihar', type='Court'), type='WORKS_FOR'), 
    Relationship(source=Node(id='S.T. No. 204 of 2010', type='Case'), target=Node(id='Addl. District and Sessions Judge-II, Katihar', type='Court'), type='HELD_IN'), 
    Relationship(source=Node(id='S.T. No. 204 of 2010', type='Case'), target=Node(id='Ranjeet Choudhary', type='Person'), type='CONCERNING'), 
    Relationship(source=Node(id='S.T. No. 204 of 2010', type='Case'), target=Node(id='Ram Kishore Chaudhary', type='Person'), type='CONCERNING'), 
    Relationship(source=Node(id='Cr. Appeal (DB) No. 117 of 2013', type='Case'), target=Node(id='Ranjeet Choudhary', type='Person'), type='CONCERNING'), 
    Relationship(source=Node(id='Cr. Appeal (DB) No. 19 of 2013', type='Case'), target=Node(id='Ram Kishore Chaudhary', type='Person'), type='CONCERNING'), 
    Relationship(source=Node(id='Falka Pothiya P.S. Case No. 26 of 2008', type='Case'), target=Node(id='Ranjeet Choudhary', type='Person'), type='CONCERNING'), 
    Relationship(source=Node(id='Falka Pothiya P.S. Case No. 26 of 2008', type='Case'), target=Node(id='Ram Kishore Chaudhary', type='Person'), type='CONCERNING'), 
    Relationship(source=Node(id='Falka Pothiya P.S. Case No. 26 of 2008', type='Case'), target=Node(id='Uday Kumar', type='Person'), type='INVESTIGATED_BY'), 
    Relationship(source=Node(id='Falka Pothiya P.S. Case No. 26 of 2008', type='Case'), target=Node(id='Informant', type='Person'), type='FILED_BY'), 
    Relationship(source=Node(id='Title Suit No. 43 of 1998', type='Case'), target=Node(id='deceased', type='Person'), type='FILED_BY'), 
    Relationship(source=Node(id='Title Suit No. 43 of 1998', type='Case'), target=Node(id='Ranjeet Choudhary', type='Person'), type='CONCERNING'), 
    Relationship(source=Node(id='Title Suit No. 43 of 1998', type='Case'), target=Node(id='brother of Ranjeet Choudhary', type='Person'), type='CONCERNING'), 
    Relationship(source=Node(id='Title Appeal No. 19 of 2006', type='Case'), target=Node(id='Ranjeet Choudhary', type='Person'), type='FILED_BY'), 
    Relationship(source=Node(id='Title Appeal No. 19 of 2006', type='Case'), target=Node(id='Additional District and Sessions Judge-I, Katihar', type='Court'), type='HELD_IN'), 
    Relationship(source=Node(id='deceased', type='Person'), target=Node(id='land dispute', type='Dispute'), type='INVOLVED_IN'), Relationship(source=Node(id='Ranjeet Choudhary', type='Person'), target=Node(id='land dispute', type='Dispute'), type='INVOLVED_IN'), 
    Relationship(source=Node(id='Ram Kishore Chaudhary', type='Person'), target=Node(id='land dispute', type='Dispute'), type='INVOLVED_IN'), 
    Relationship(source=Node(id='Informant', type='Person'), target=Node(id='land dispute', type='Dispute'), type='INVOLVED_IN'), Relationship(source=Node(id='deceased', type='Person'), target=Node(id='fire arm injury', type='Injury'), type='DIED_FROM'), Relationship(source=Node(id='deceased', type='Person'), target=Node(id='fire arm injury', type='Injury'), type='RECEIVED_INJURY'), 
    Relationship(source=Node(id='deceased', type='Person'), target=Node(id='wound of exit', type='Injury'), type='RECEIVED_INJURY'), Relationship(source=Node(id='deceased', type='Person'), target=Node(id='Mantaria Bandh', type='Location'), type='DIED_AT'), Relationship(source=Node(id='deceased', type='Person'), target=Node(id='palani', type='Structure'), type='CONSTRUCTING'), Relationship(source=Node(id='Informant', type='Person'), target=Node(id='palani', type='Structure'), type='CONSTRUCTING'), Relationship(source=Node(id='Ranjeet Choudhary', type='Person'), target=Node(id='country made rifle', type='Weapon'), type='USED'), 
    Relationship(source=Node(id='Ram Kishore Chaudhary', type='Person'), target=Node(id='weapon', type='Weapon'), type='USED'), Relationship(source=Node(id='Ranjeet Choudhary', type='Person'), target=Node(id='Ram Kishore Chaudhary', type='Person'), type='ORDERED_BY'), 
    Relationship(source=Node(id='Ranjeet Choudhary', type='Person'), target=Node(id='deceased', type='Person'), type='FIRED_AT'), Relationship(source=Node(id='Ranjeet Choudhary', type='Person'), target=Node(id='Suro Devi', type='Person'), type='FIRED_AT'), Relationship(source=Node(id='Suro Devi', type='Person'), target=Node(id='fire arm injury', type='Injury'), type='RECEIVED_INJURY'), 
    Relationship(source=Node(id='Suro Devi', type='Person'), target=Node(id='deceased', type='Person'), type='MOTHER_OF'), Relationship(source=Node(id='Suro Devi', type='Person'), target=Node(id='Informant', type='Person'), type='MOTHER_OF'), Relationship(source=Node(id='Beena Devi', type='Person'), target=Node(id='Satish Kumar Yadav', type='Person'), type='MOTHER_OF'), Relationship(source=Node(id='Uday Kumar', type='Person'), target=Node(id='fardbeyan', type='Document'), type='RECORDED'), Relationship(source=Node(id='Uday Kumar', type='Person'), target=Node(id='death inquest report', type='Document'), type='PREPARED'), 
    Relationship(source=Node(id='Uday Kumar', type='Person'), target=Node(id='statement of Satish Kumar Yadav', type='Document'), type='RECORDED'), 
    Relationship(source=Node(id='Uday Kumar', type='Person'), target=Node(id='statement of Uro Devi', type='Document'), type='RECORDED'), 
    Relationship(source=Node(id='Uday Kumar', type='Person'), target=Node(id='statement of Beena Devi', type='Document'), type='RECORDED'), 
    Relationship(source=Node(id='Uday Kumar', type='Person'), target=Node(id='statement of Satyanarain Yadav', type='Document'), type='RECORDED'), 
    Relationship(source=Node(id='Uday Kumar', type='Person'), target=Node(id='Informant', type='Person'), type='MET'), 
    Relationship(source=Node(id='Uday Kumar', type='Person'), target=Node(id='occurrence', type='Event'), type='HEARD_ABOUT'), Relationship(source=Node(id='Uday Kumar', type='Person'), target=Node(id='dead body of the deceased', type='Body'), type='FOUND'), 
    Relationship(source=Node(id='Uday Kumar', type='Person'), target=Node(id='blood mark', type='Mark'), type='DID_NOT_FIND'), Relationship(source=Node(id='Uday Kumar', type='Person'), target=Node(id='shell of cartridges', type='Object'), type='DID_NOT_FIND'), 
    Relationship(source=Node(id='Informant', type='Person'), target=Node(id='assault on his brother', type='Event'), type='SAW'), Relationship(source=Node(id='Informant', type='Person'), target=Node(id='place of occurrence', type='Location'), type='FLED_FROM'), 
    Relationship(source=Node(id='Informant', type='Person'), target=Node(id='Dumar Chowk', type='Location'), type='WENT_TO'), Relationship(source=Node(id='Informant', type='Person'), target=Node(id='Police Station', type='Location'), type='WENT_TO'), Relationship(source=Node(id='Informant', type='Person'), target=Node(id='inquest report', type='Document'), type='SIGNED'), Relationship(source=Node(id='Anil Kumar', type='Person'), target=Node(id='inquest report', type='Document'), type='SIGNED'), Relationship(source=Node(id='F.I.R.', type='Document'), target=Node(id='17.07.2008', type='Date'), type='REGISTERED_ON'), Relationship(source=Node(id='F.I.R.', type='Document'), target=Node(id='Court', type='Court'), type='DISPATCH_TO'), 
    Relationship(source=Node(id='F.I.R.', type='Document'), target=Node(id='Magistrate', type='Person'), type='SEEN_BY'), 
    Relationship(source=Node(id='F.I.R.', type='Document'), target=Node(id='Magistrate', type='Person'), type='SIGNED_BY'), Relationship(source=Node(id='Magistrate', type='Person'), target=Node(id='F.I.R.', type='Document'), type='SEEN'), 
    Relationship(source=Node(id='Magistrate', type='Person'), target=Node(id='F.I.R.', type='Document'), type='SIGNED')
]

In [ ]:

abc = [
    GraphDocument(
        nodes=[Node(id='deceased', type='Person'), Node(id='Ram Kishore Chaudhary', type='Person'), ],

        relationships=[Relationship(source=Node(id='Ranjeet Choudhary', type='Person'), target=Node(id='Section 302 IPC', 
            type='Law'), type='CONVICTED_OF'), ], 

        source=Document(
            metadata={'source': 'html_data\\1197362.htm', 'title': ''}, 
            page_content='@JUDGMENTTAG-JUDGMENT\nMr. Kishore Kumar Mandal, J. (Oral)—These two appeals, each filed by one appellant, are directed against the judgment of conviction dated 19.12.2012 and the order of sentence dated 24.12.2012, passed by learned Addl. District and Sessions e.\n'
        )
    )
]

In [ ]:
Node = ['Case', 'Court', 'ApplealedCase', 'CourtData', 'SummaryData', 'CitationsData']
Relationships = ['Appeal_from', 'Appeal_to', 'Is_case', 'Has_data', 'Neighbouring_data']
NodeProperties = {
    'Case':['case_name', 'case_no', 'result', 'overruled', 'overruled_by'], 
    'Court':['courst_name', 'Court_abbreviation'], 
    'ApplealedCase':['case_name', 'case_no', 'result', 'overruled', 'overruled_by'], 
    'CourtData':['type_of_case', 'petetioner', 'respondent', 'coram', 'petetioner_counsel', 'respondent_counsel', 'act', 'bench', 'dated', 'reportable', 'case_no'], 
    'SummaryData':['case_no', 'evidence', 'conclusion', 'courts_reasoning', 'precedent_analysis', 'legal_analysis',  'respondents_arguments', 'petitioners_arguments', 'issues', 'facts', 'summary'], 
    'CitationsData':['citations', 'cited', 'headnotes', 'case_referred', 'keywords','case_no']
}
Relationship_between = [
    {
        'source':'Court',
        'target':'Case',
        'relationship':'Is_case'
    },
    {
        'source':'Court',
        'target':'AppealedCase',
        'relationship':'Is_case'
    },
    {
        'source':'AppealedCase',
        'target':'Case',
        'relationship':'Appeal_from'
    },
    {
        'source':'Case',
        'target':'AppealedCase',
        'relationship':'Appeal_to'
    },
    {
        'source':'Case',
        'target':'CourtData',
        'relationship':'Has_data'
    },
    {
        'source':'Case',
        'target':'SummaryData',
        'relationship':'Has_data'
    },
    {
        'source':'Case',
        'target':'CitationsData',
        'relationship':'Has_data'
    },
    {
        'source':'CitationsData',
        'target':'CourtData',
        'relationship':'Neighbouring_data'
    },
    {
        'source':'CitationsData',
        'target':'SummaryData',
        'relationship':'Neighbouring_data'
    },
    {
        'source':'CourtData',
        'target':'CitationsData',
        'relationship':'Neighbouring_data'
    },
    {
        'source':'SummaryData',
        'target':'CitationsData',
        'relationship':'Neighbouring_data'
    },
    {
        'source':'CourtData',
        'target':'SummaryData',
        'relationship':'Neighbouring_data'
    },
    {
        'source':'SummaryData',
        'target':'CourtData',
        'relationship':'Neighbouring_data'
    },
]



In [1]:
from langchain import hub

prompt = hub.pull("hwchase17/react")

c:\Users\admin\Projects\Assignments\knowledge_graph\.venv\Lib\site-packages\langsmith\client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
c:\Users\admin\Projects\Assignments\knowledge_graph\.venv\Lib\site-packages\langsmith\client.py:5489: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


In [2]:
import os
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from neomodel import db, config


load_dotenv()
google_api_key = os.getenv('GOOGLE_API_KEY')
url = os.getenv('NEO4J_URI')
username = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')
neo4j_url = os.getenv('NEO4J_URI_0')
database = 'newdb'

c:\Users\admin\Projects\Assignments\knowledge_graph\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
graph = Neo4jGraph(url=url, username=username, password=password, database=database)
print(graph.schema)

Node properties:
Court {court_name: STRING, court_abbreviation: STRING}
Case {case_name: STRING, result: STRING, case_no: STRING, overruled: STRING, overruled_by: STRING}
CourtData {reportable: STRING, petitioner: STRING, case_type: STRING, respondent: STRING, bench: STRING, coram: STRING, dated: STRING, case_no: STRING, petitioner_counsel: STRING, respondent_counsel: STRING, act: STRING}
SummaryData {petitioners_arguments: STRING, courts_reasoning: STRING, summary: STRING, evidence: STRING, respondents_arguments: STRING, issues: STRING, facts: STRING, case_no: STRING, conclusion: STRING, legal_analysis: STRING, precedent_analysis: STRING}
CitationsData {case_no: STRING, cited: STRING, citations: STRING, case_referred: STRING, keywords: STRING, headnotes: STRING}
Relationship properties:

The relationships:
(:Court)-[:Is_case]->(:Case)
(:Case)-[:Has_data]->(:CitationsData)
(:Case)-[:Has_data]->(:SummaryData)
(:Case)-[:Has_data]->(:CourtData)
(:CourtData)-[:Neighbouring_data]->(:Summary